In [42]:
import pandas as pd
import numpy as np
import json
import re
import spacy # NLP Preprocessor
spacy.prefer_gpu()

False

In [43]:
def get_data_from_json(path: str) -> pd.DataFrame:
    with open(path, "r") as json_file:
        json_lines = (line for line in json_file)
        rows = (
            {
                "text": data["text"],
                "comment": comment["text"],
                "score": comment["score"]
            }
            for line in json_lines
            for data in [json.loads(line)]
            for comment in data["comments"]
        )
        return pd.DataFrame(rows, columns=["text", "comment", "score"])



In [44]:
dataset_path = './datasets/ranking_train.jsonl'
df = get_data_from_json(dataset_path)

In [45]:
test = df.sample(10) 

In [46]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(test.iloc[0].comment)

In [59]:
def remove_stops(doc):
    """
    Удаляет часто встречающиеся слова
    """
    wo_stops = []
    for token in doc:
        if not token.is_stop:
            wo_stops.append(token.text)

    wo_stops = " ".join(wo_stops)
    return wo_stops


def remove_specials(wo_stops):
    """
    Удаляет cпециальные симловы оставляя только буквы
    """
    clean_text = re.sub('[^A-Za-z0-9]+', ' ', wo_stops)
    clean_text = " ".join(
        [text for text in clean_text.split(' ') if len(text) > 2])
    return clean_text

def extract_feats(doc):
    """
    Преобразовать текст к массиву слов
    """
    feats = []
    for token in doc:
        if token.pos_ in ['NOUN','VERB']:
            feats.append(token.text)
    return feats

In [67]:
doc = nlp(test.iloc[1].comment)

In [57]:
wo_stops = remove_stops(doc)
clean_text = remove_specials(wo_stops)